In [1]:

word_dict = {
"isha'": "coyote",
"isha'pugu": "dog",
"kidi'": "cat",
"pugu": "horse",
"wai": "rice",
"tüba": "pinenuts",
"maishibü": "corn",
"paya": "water",
"payahuupü": "river",
"katünu": "chair",
"toyabi": "mountain",
"tuunapi": "food",
"pasohobü": "tree",
"nobi": "house",
"toni": "wickiup",
"apo": "cup",
"küna": "wood",
"tübbi": "rock",
"tabuutsi'": "cottontail",
"kamü": "jackrabbit",
"aaponu'": "apple",
"tüsüga": "weasle",
"mukita": "lizard",
"wo'ada": "mosquito",
"wükada": "bird snake",
"wo'abi": "worm",
"aingwü": "squirrel",
"tsiipa": "bird",
"tüwoobü": "earth",
"koopi'": "coffee",
"pahabichi": "bear",
"pagwi": "fish",
"kwadzi": "tail",
'ii': 'proximal',
'uu': 'distal',
'katü': 'sit',
'üwi': 'sleep',
'kwisha\'i': 'sneeze',
'poyoha': 'run',
'mia': 'go',
'hukaw̃ia': 'walk',
'wünü': 'stand',
'habi': 'lie down',
'yadoha': 'talk',
'kwatsa\'i': 'fall',
'waakü': 'work',
'wükihaa': 'smile',
'hubiadu': 'sing',
'nishua\'i': 'laugh him',
'tsibui': 'climb',
'tübinohi': 'play',
'yotsi': 'fly',
'nüga': 'dance',
'pahabi': 'swim',
'tünia': 'read',
'tümui': 'write',
'tsiipe\'i': 'chirp',
'eika': 'proximal',
'oka': 'distal',
'i': 'me',
'u': 'him',
'ui': 'them',
'ma': 'him',
'mai': 'them',
'a': 'him',
'ai': 'them',
'ni': 'us',
'tei': 'us',
'ta': 'us',
'ü': 'you',
'üi': 'you',
'tüka': 'eat',
'puni': 'see',
'hibi': 'drink',
'naka': 'hear',
'kwana': 'smell',
'kwati': 'hit',
'yadohi': 'talk to',
'naki': 'chase',
'tsibui': 'climb',
'sawa': 'cook',
'tama\'i': 'find',
'nia': 'read',
'mui': 'write',
'nobini': 'visit',
'gaa-wei': 'going to',
'pü': 'have x-ed',
}


In [2]:
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModelForSeq2SeqLM
# tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = BertModel.from_pretrained('bert-base-multilingual-cased')
model_checkpoint = "Helsinki-NLP/opus-mt-mul-en"
# model_checkpoint="google-t5/t5-large"
# model_checkpoint="/home/remote/dev/llm/Helsinki-NLP/opus-mt-mul-en_-single-word-fine-tune/"
# model_checkpoint = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

/home/remote/miniconda3/envs/kubi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/remote/miniconda3/envs/kubi/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/remote/miniconda3/envs/kubi/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [3]:
import pandas as pd


df = pd.read_csv('translations.csv')

In [4]:
training_corpus = [df["sentence"][i: i + 10].values for i in range(0, len(df["sentence"]), 10)]


In [5]:
training_corpus[0][0]

'tei-zawa-ku ihi'

In [6]:
tokenizer.tokenize(training_corpus[0][0])

['▁tei', '-', 'za', 'wa', '-', 'ku', '▁i', 'hi']

In [5]:
new_tokenizer = tokenizer#.train_new_from_iterator(training_corpus, 129547)

In [8]:
new_tokenizer.tokenize(training_corpus[0][1])

['▁i',
 'sha',
 "'",
 '-',
 'u',
 'u',
 '▁',
 'tü',
 'ba',
 '-',
 'ne',
 'ika',
 '▁ai',
 '-',
 'hi',
 'bi',
 '-',
 'p',
 'ü']

In [8]:
max_length = 256
inputs = list(word_dict.values()) #df['sentence'].values.tolist()
targets = list(word_dict.keys()) #df['translation'].values.tolist()

input_data = []
for k, v in word_dict.items():
    input_data.append(
        {
            'src': v.strip(),
            'target': k.strip()
        }
    )

In [9]:
import pandas as pd


df_dataset = pd.DataFrame(input_data)
df_dataset

,src,target
0,coyote,isha'
1,dog,isha'pugu
2,cat,kidi'
3,horse,pugu
4,rice,wai
...,...,...
81,read,nia
82,write,mui
83,visit,nobini
84,going to,gaa-wei


In [10]:
prefix="Translate Paiute to English: "
def preprocess_data(data):

    model_inputs = new_tokenizer(
    data['src'], text_target=data['target'], max_length=max_length, truncation=True
    )

    return model_inputs

def preproc_t5(data):
    inputs = [prefix + example for example in data["sentence"]]
    targets = [example for example in data["translation"]]
    model_inputs = new_tokenizer(inputs, max_length=256, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = new_tokenizer(targets, max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [11]:
from datasets import Dataset
# data = df[['sentence', 'translation']]
_dataset = Dataset.from_pandas(df_dataset)


In [12]:
tokenized_dataset = _dataset.map(    preprocess_data,
    batched=True, 
     remove_columns=_dataset.column_names,
)

Map: 100%|██████████| 86/86 [00:00<00:00, 7110.39 examples/s]


In [13]:
from transformers import TrainingArguments, Trainer



In [14]:
import evaluate

metric = evaluate.load("sacrebleu")

In [15]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = new_tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, new_tokenizer.pad_token_id)
    decoded_labels = new_tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [16]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/home/remote/miniconda3/envs/kubi/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
new_words = list(word_dict.keys())
new_tokenizer.add_tokens(new_words)
tokenizer.add_tokens(new_words)
print(len(tokenizer))  # 28997


model.resize_token_embeddings(len(tokenizer)) 

64243


Embedding(64243, 512)

In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(new_tokenizer, model=model)

In [ ]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [18]:

from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"{model_checkpoint}-word-fine-tune-en-to-pt-1k",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.001,
    save_total_limit=3,
    num_train_epochs=1000,
    predict_with_generate=True,
    fp16=True,
)
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=new_tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics
)


trainer.train()
# print(trainer.evaluate(max_length=max_length))

Step,Training Loss
500,1.089300
1000,0.060700
1500,0.057000
2000,0.054600
2500,0.053000
3000,0.051600


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[64171]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[64171]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=3000, training_loss=0.22772036043802898, metrics={'train_runtime': 1528.7511, 'train_samples_per_second': 56.255, 'train_steps_per_second': 1.962, 'total_flos': 126416536731648.0, 'train_loss': 0.22772036043802898, 'epoch': 1000.0})

In [34]:
del model

In [19]:
model = trainer.model_wrapped

In [ ]:
df.iloc[8]

sentence            isha'-ii tübbi-neika mai-w̃ui-gaa-wei
translation    This coyote is going to write these rocks.
label                                                   1
Name: 8, dtype: object

In [ ]:
og_model = trainer.model

In [19]:
input_text = training_corpus[0][8]
input_text

"isha'-ii tübbi-neika mai-w̃ui-gaa-wei"

In [46]:
input_text="This coyote is going to write these rocks."

In [44]:
input_text="tuunapi-noka ihiw̃a u-yadohi-ti"

In [25]:
input_text="Translate Paiute to English: isha'pugu-neika mahu ma-hibi-dü"

In [31]:
inputs = new_tokenizer(input_text,  return_tensors="pt").input_ids.to("cuda")

In [20]:
model.eval()

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(64172, 512, padding_idx=64171)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(64172, 512, padding_idx=64171)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [37]:
input_text="tuunapi-noka ihiw̃a u-yadohi-ti"

In [76]:
input_text="isha'pugu-neika isha'pugu-noka mai-w̃ui-gaa-wei"
input_text="pahabichi-ii tuunapi-noka u-düka-ti"
input_text="pagwi-uu tsibui-ti"
input_text="Those fish are climbing."
input_text="isha'pugu-uu tübinohi-ku."
input_text="That dog played."

In [ ]:
"isha'pugu": "dog",
"kidi'": "cat",
"pugu": "horse",
"wai": "rice",
"tüba": "pinenuts",
"maishibü": "corn",
"paya": "water",
"payahuupü": "river",
"katünu": "chair",
"toyabi": "mountain",
"tuunapi": "food",
"pasohobü": "tree",
"nobi": "house",
"toni": "wickiup",
"apo": "cup",
"küna": "wood",
"tübbi": "rock",
"tabuutsi'": "cottontail",
"kamü": "jackrabbit",
"aaponu'": "apple",
"tüsüga": "weasle",
"mukita": "lizard",
"wo'ada": "mosquito",
"wükada": "bird snake",
"wo'abi": "worm",
"aingwü": "squirrel",
"tsiipa": "bird",
"tüwoobü": "earth",
"koopi'": "coffee",
"pahabichi": "bear",
"pagwi": "fish",
"kwadzi": "tail",
'ii': 'proximal',
'uu': 'distal',
'katü': 'sit',
'üwi': 'sleep',
'kwisha\'i': 'sneeze',
'poyoha': 'run',
'mia': 'go',
'hukaw̃ia': 'walk',
'wünü': 'stand',

In [23]:
input_text="mia"

In [24]:
inputs = new_tokenizer(input_text,  return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(inputs)#,max_length=512, min_length=8, top_p=0.9, do_sample=True)
print(tokenizer.decode(outputs[0]))
print(new_tokenizer.decode(outputs[0]))




<pad> go</s>
<pad> go</s>
